In [15]:
#imports 
import json
import requests
import pandas as pd
import numpy as np
import matplotlib as plt
import pprint as pprint

In [16]:
# Raw geo data file to be saved and pulled from
rawGeoData = "../data/rawData/es_geo_raw_dataframe.csv"

# cleaned data file to be saved
cleanData = "../data/cleanData/es_clean_data.csv"

# path csv file location
pathRawData = "../data/rawData/"
pathCleanData ="../datacleanData/"

In [17]:
#calling the csv raw data
geo_csv_file = pd.read_csv(rawGeoData,index_col=0)
geo_csv_file.head()

,Job Title,Category,Company Name,Location,Lat,Lng,LatLng,geocode_data,city,state,country
Job Posting ID,,,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",Clayton,New York,United States
1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",Sunbury,Ohio,United States
1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,Cheshire,Connecticut,United States
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",Marion,Illinois,United States
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",Cromwell,Connecticut,United States


In [18]:
#Maiking a copy of Geo Data
geo_csv_file_copy = geo_csv_file.copy()
geo_csv_file_copy.count()

Job Title       983
Category        983
Company Name    983
Location        983
Lat             983
Lng             983
LatLng          983
geocode_data    983
city            983
state           983
country         983
dtype: int64

In [19]:
#Dropping any rows that are missing 
geo_data = geo_csv_file_copy.dropna()
geo_data.count()

Job Title       983
Category        983
Company Name    983
Location        983
Lat             983
Lng             983
LatLng          983
geocode_data    983
city            983
state           983
country         983
dtype: int64

In [20]:
#Calling only Rows that have United states 
dropping_non_us = geo_data[geo_data.country == 'United States']
dropping_non_us.count()

Job Title       977
Category        977
Company Name    977
Location        977
Lat             977
Lng             977
LatLng          977
geocode_data    977
city            977
state           977
country         977
dtype: int64

In [21]:
#If there are dupicates Job ID's dorp them and keep the first
clean_data = dropping_non_us.loc[~dropping_non_us.index.duplicated(keep='first')]
clean_data.count()

Job Title       977
Category        977
Company Name    977
Location        977
Lat             977
Lng             977
LatLng          977
geocode_data    977
city            977
state           977
country         977
dtype: int64

In [22]:
#Check how many different categories you have
checking_cat = clean_data['Category'].unique()
list(checking_cat)

['Healthcare & Nursing Jobs',
 'Other/General Jobs',
 'Unknown',
 'Customer Services Jobs',
 'Sales Jobs',
 'Retail Jobs',
 'Logistics & Warehouse Jobs',
 'Accounting & Finance Jobs']

In [23]:
#Creating columns that have unknown category, define those unknown categories and append to final_c column

#empty column to store the final Category column
clean_data['final_c'] = np.NaN

#Logistics & Warehouse Jobs
warehouse = clean_data[clean_data['Job Title'].str.contains("Warehous")]
clean_data['warehouse_col'] = warehouse['Category'].replace('Unknown', 'Logistics & Warehouse Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['warehouse_col'])

#Customer Services Jobs
shopper = clean_data[clean_data['Job Title'].str.contains("Shopp")]
clean_data['shopper_col'] = shopper['Category'].replace('Unknown', 'Customer Services Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['shopper_col'])

##Healthcare & Nursing Job##

#Physcician
physician = clean_data[clean_data['Job Title'].str.contains("Physician")]
clean_data['physician_col'] = physician['Category'].replace('Unknown', 'Healthcare & Nursing Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['physician_col'])

#Nurse
nurse = clean_data[clean_data['Job Title'].str.contains("Nurs")]
clean_data['nurse_col'] = nurse['Category'].replace('Unknown', 'Healthcare & Nursing Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['nurse_col'])

#Teaching Jobs
teach = clean_data[clean_data['Job Title'].str.contains("Teach")]
clean_data['teacher_col'] = teach['Category'].replace('Unknown', 'Teaching Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['teacher_col'])

#Final column Category to append into final_c column
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['Category'])

clean_data

,Job Title,Category,Company Name,Location,Lat,Lng,LatLng,geocode_data,city,state,country,final_c,warehouse_col,shopper_col,physician_col,nurse_col,teacher_col
Job Posting ID,,,,,,,,,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",Clayton,New York,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",Sunbury,Ohio,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,Cheshire,Connecticut,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",Marion,Illinois,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",Cromwell,Connecticut,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320153715,Physician: Interventional Cardiology Opportuni...,Healthcare & Nursing Jobs,CompHealth,"Durant, Bryan County",33.993986,-96.370824,"33.993986,-96.370824","{'address_components': [{'long_name': '507', '...",Durant,Oklahoma,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1338909279,Warehouse Technician - Up to $20.40 HR Salary,Unknown,Amazon,"Cupertino, Santa Clara County",37.322998,-122.032182,"37.322998,-122.032182",{'address_components': [{'long_name': 'Stevens...,Cupertino,California,United States,Logistics & Warehouse Jobs,Logistics & Warehouse Jobs,NaN,NaN,NaN,NaN
1312808260,Physician: Cardiothoracic Surgeon Position in ...,Healthcare & Nursing Jobs,CompHealth,"Wisconsin, US",44.391021,-89.773080,"44.391021,-89.77308000000002","{'address_components': [{'long_name': '312', '...",Wisconsin Rapids,Wisconsin,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN


In [24]:
#Drop rows unknown attributes in column final_c

# Get names of indexes for which column Unknown
drop_unknown = clean_data[ clean_data['final_c'] == 'Unknown' ].index
# Delete these row indexes from dataFrame
clean_data.drop(drop_unknown , inplace=True)
clean_data

,Job Title,Category,Company Name,Location,Lat,Lng,LatLng,geocode_data,city,state,country,final_c,warehouse_col,shopper_col,physician_col,nurse_col,teacher_col
Job Posting ID,,,,,,,,,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",Clayton,New York,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",Sunbury,Ohio,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,Cheshire,Connecticut,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",Marion,Illinois,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",Cromwell,Connecticut,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338900671,Seasonal Warehousing Worker,Unknown,Amazon,"Oakland, Alameda County",37.772927,-122.214332,"37.772927,-122.21433200000001",{'address_components': [{'long_name': 'Oakland...,Oakland,California,United States,Logistics & Warehouse Jobs,Logistics & Warehouse Jobs,NaN,NaN,NaN,NaN
1320153715,Physician: Interventional Cardiology Opportuni...,Healthcare & Nursing Jobs,CompHealth,"Durant, Bryan County",33.993986,-96.370824,"33.993986,-96.370824","{'address_components': [{'long_name': '507', '...",Durant,Oklahoma,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1338909279,Warehouse Technician - Up to $20.40 HR Salary,Unknown,Amazon,"Cupertino, Santa Clara County",37.322998,-122.032182,"37.322998,-122.032182",{'address_components': [{'long_name': 'Stevens...,Cupertino,California,United States,Logistics & Warehouse Jobs,Logistics & Warehouse Jobs,NaN,NaN,NaN,NaN


In [25]:
#Drop unwanted columns
dropped_columns = clean_data.drop(columns=['Category','warehouse_col', 'Location','shopper_col','physician_col','nurse_col','teacher_col','geocode_data'])
dropped_columns.head()

,Job Title,Company Name,Lat,Lng,LatLng,city,state,country,final_c
Job Posting ID,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,CompHealth,44.239491,-76.085776,"44.239490999999994,-76.085776",Clayton,New York,United States,Healthcare & Nursing Jobs
1312805901,Physician: Urology Need in Western Ohio with L...,CompHealth,40.305232,-82.824785,"40.305232000000004,-82.82478499999998",Sunbury,Ohio,United States,Healthcare & Nursing Jobs
1328241931,Physician: Internal Medicine Physician Job in ...,CompHealth,41.508367,-72.910620,"41.508367,-72.91062",Cheshire,Connecticut,United States,Healthcare & Nursing Jobs
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,CompHealth,37.730605,-88.933126,"37.730605,-88.933126",Marion,Illinois,United States,Healthcare & Nursing Jobs
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",CompHealth,41.601859,-72.657316,"41.60185900000001,-72.65731600000002",Cromwell,Connecticut,United States,Healthcare & Nursing Jobs


In [26]:
#Re-arrange columns
arrange_col_data = dropped_columns[['Job Title','final_c','Company Name','city','state','country','Lat','Lng','LatLng']]
arrange_col_data.head()

,Job Title,final_c,Company Name,city,state,country,Lat,Lng,LatLng
Job Posting ID,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,Clayton,New York,United States,44.239491,-76.085776,"44.239490999999994,-76.085776"
1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,Sunbury,Ohio,United States,40.305232,-82.824785,"40.305232000000004,-82.82478499999998"
1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,Cheshire,Connecticut,United States,41.508367,-72.910620,"41.508367,-72.91062"
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,Marion,Illinois,United States,37.730605,-88.933126,"37.730605,-88.933126"
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,Cromwell,Connecticut,United States,41.601859,-72.657316,"41.60185900000001,-72.65731600000002"


In [27]:
#Renames column Names
cleansed_data = arrange_col_data.rename(columns={"final_c": "Category","city":"City","state":"State","country":"Country"})
cleansed_data.head()

,Job Title,Category,Company Name,City,State,Country,Lat,Lng,LatLng
Job Posting ID,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,Clayton,New York,United States,44.239491,-76.085776,"44.239490999999994,-76.085776"
1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,Sunbury,Ohio,United States,40.305232,-82.824785,"40.305232000000004,-82.82478499999998"
1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,Cheshire,Connecticut,United States,41.508367,-72.910620,"41.508367,-72.91062"
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,Marion,Illinois,United States,37.730605,-88.933126,"37.730605,-88.933126"
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,Cromwell,Connecticut,United States,41.601859,-72.657316,"41.60185900000001,-72.65731600000002"


In [30]:
#Save cleansed data to csv
cleansed_data.to_csv(cleanData)